In [ ]:
# currently required as a workaround to https://github.com/brightway-lca/brightway-live/issues/10
import os
os.environ["BRIGHTWAY_DIR"] = "/tmp/"

In [ ]:
import bw2data as bd
import bw2io as bi
import peewee

In [ ]:
# seems to throw an error currently (version (0, 9, 'DEV26')), but correctly imports the database 
bi.add_example_database(searchable=True, overwrite=True)

In [ ]:
db = bd.Database("Mobility example")

In [ ]:
# actually select the peewee sqlite3 database, which is hidden behind layers of Brightway class inheritance
from bw2data.backends import sqlite3_lci_db
sqldb: peewee.SqliteDatabase = sqlite3_lci_db.db
cursor = sqldb.cursor()

In [ ]:
# list all tables of the database
# https://stackoverflow.com/a/10746045
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
# list all columns of a database table
cursor.execute("PRAGMA table_info('activitydataset')")
columns = [column[1] for column in cursor.fetchall()]
print(columns)

In [ ]:
rand = cursor.execute("SELECT * FROM activitydataset ORDER BY RANDOM() LIMIT 1").fetchone()
rand

In [ ]:
# add an fts5 search table
# https://saraswatmks.github.io/2020/04/sqlite-fts-search-queries.html
cursor.execute(
    """
    create virtual table search_table_activitydataset
    using fts5(
        id,
        name,
        product,
        type,
        tokenize="porter unicode61");
    )
    """
)

In [ ]:
# populate fts5 table with data
cursor.execute(
    """
    INSERT INTO search_table_activitydataset (id, name, product, type) 
    SELECT id, name, product, type FROM activitydataset
    """
)

In [ ]:
# check if any data made it into the fts5 table
count = cursor.execute(f"SELECT COUNT(*) FROM search_table_activitydataset").fetchone()[0]
count

In [ ]:
# run a search query against the fts5 table
res = cur.execute(
    """
    select *, rank
    from search_table_activitydataset
    where name MATCH "car"
    ORDER BY rank
    limit 3
    """
).fetchall()
res